In [0]:
%md
用现有的  s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-cum.v1/fact/   和 s3://b2c-prod-data-pipeline-unified-store-free/unified/store.app.v1/dimension  full outer join 看下，app_id,  publisher 对应关系，找出app_id ， publisher 互相对不上的 ，数量， 以及sample
在df-datatech-delivery channel里 找Data collection 同事确认下

In [0]:

aa = spark.read.parquet("s3://b2c-prod-data-pipeline-qa/aa.store.app-est-cum.v1/store_cum_completeness_result/").collect()
print len(aa)

In [0]:
%%sh
aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-cum.v1/fact/

In [0]:
%%sh
aws s3 ls s3://b2c-prod-data-pipeline-unified-store-free/unified/store.app.v1/dimension/

In [0]:

df1 = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-free/unified/store.app.v1/dimension/")
df1.show(10, True, True)

In [0]:

df1 = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-free/unified/store.app.v1/dimension/").select("id", "publisher_id")
df1.show(10)

In [0]:

df2 = spark.read.format('delta').load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-cum.v1/fact/date=2020-04-18/")
df2.show(10)

In [0]:

df1.where("id='284942940'").show(10)

In [0]:

df2.where("app_id='20600001642518'").show(10)

In [0]:

pub_df = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-free/unified/store.app.v1/dimension/").select('id', 'publisher_id')
cum_df = spark.read.format('delta').load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-cum.v1/fact/date=2020-04-18/")

pub_df.createOrReplaceTempView('pub_tbl')
cum_df.createOrReplaceTempView('cum_tbl')

join_df = spark.sql(
    "SELECT * "
    "FROM pub_tbl FULL OUTER JOIN cum_tbl ON pub_tbl.id=cum_tbl.app_id "
    "WHERE id is NULL OR app_id is NULL"
    )
join_df.show(10)


In [0]:

print join_df.count()

In [0]:

print pub_df.count()
print cum_df.count()

In [0]:

pub_df = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-free/unified/store.app.v1/dimension/").select('id', 'publisher_id')
cum_df = spark.read.format('delta').load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-cum.v1/fact/date=2020-04-18/")

pub_df.createOrReplaceTempView('pub_tbl')
cum_df.createOrReplaceTempView('cum_tbl')
# join_df2 = spark.sql(
#     "SELECT *"
#     "FROM pub_tbl FULL OUTER JOIN cum_tbl ON pub_tbl.id=cum_tbl.app_id "
#     "WHERE id is NULL"
#     )
# print join_df2.select('app_id').distinct().count()
spark.sql(
    "SELECT count(distinct app_id) "
    "FROM cum_tbl LEFT JOIN pub_tbl ON pub_tbl.id=cum_tbl.app_id "
    "WHERE publisher_id is NULL"
    ).show(10)


In [0]:

spark.sql(
    "SELECT DISTINCT app_id "
    "FROM cum_tbl LEFT JOIN pub_tbl ON pub_tbl.id=cum_tbl.app_id "
    "WHERE publisher_id is NULL "
    "LIMIT 10"
    ).show(10)

In [0]:

spark.sql(
    "SELECT * "
    "FROM cum_tbl LEFT JOIN pub_tbl ON pub_tbl.id=cum_tbl.app_id "
    "WHERE publisher_id is NULL "
    "LIMIT 10"
    ).show(10)

In [0]:

spark.sql(
    "SELECT count(*) "
    "FROM cum_tbl JOIN pub_tbl ON pub_tbl.id=cum_tbl.app_id "
    "WHERE publisher_id is NULL"
    ).show(10)

In [0]:

pub_df1 = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-free/unified/store.app.v1/dimension/")
cum_df1 = spark.read.format('delta').load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-cum.v1/fact/date=2020-04-18/")

pub_df1.createOrReplaceTempView('pub_tbl1')
cum_df1.createOrReplaceTempView('cum_tbl1')

spark.sql(
    "SELECT * "
    "FROM cum_tbl1 JOIN pub_tbl1 ON pub_tbl1.id=cum_tbl1.app_id "
    "WHERE publisher_id is NULL "
    "LIMIT 10"
    ).show(10)

In [0]:

groupby_date = join_df2.groupby("date").count()
print groupby_date.count()

In [0]:

print join_df2.distinct().count()
# groupby_date = join_df2.groupby("date").count()
# print groupby_date.count()

In [0]:

join_df3 = spark.sql(
    "SELECT *"
    "FROM pub_tbl FULL OUTER JOIN cum_tbl ON pub_tbl.id=cum_tbl.app_id "
    "WHERE app_id is NULL"
    )
print join_df3.select('id').distinct().count()
join_df3.show(10)

In [0]:

spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-free/unified/store.app.v1/dimension/").where("id=95284").show()

In [0]:

pub_df2 = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-free/unified/store.app.v1/dimension/").select('id', 'publisher_id')
cum_df2 = spark.read.format('delta').load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-cum.v1/fact/date=2020-04-18/").select('app_id')
print pub_df2.distinct().count()
print cum_df2.distinct().count()

In [0]:

join_df.createOrReplaceTempView('join_tbl')
spark.sql("SELECT * FROM join_tbl ")

In [0]:

select_df = join_df.select("app_id", "id", "publisher_id")
select_df.show(10)

In [0]:

join_df.show(10, True, True)

In [0]:
%%sh
PGPASSWORD='TMV!PYT02X*w' psql -h 54.210.244.2 -p 5433 -U app_tomcat -d aa_android << EOF
-- \d app;
-- SELECT id, name, company FROM app LIMIT 1;
SELECT id, name, company FROM app WHERE id=383004611 LIMIT 1;
EOF

In [0]:
%md
+----+------------+------------------+----------+-----------+------------+----------+-------+-----------------+-----------------+-----------+---------------+
|  id|publisher_id|       _identifier|      date|device_code|country_code|    app_id|revenue|paid_app_download|free_app_download|revenue_iap|revenue_non_iap|
+----+------------+------------------+----------+-----------+------------+----------+-------+-----------------+-----------------+-----------+---------------+
|null|        null|120200421085053706|2020-04-18|  ios-phone|          US| 441004607|   null|             null|               32|       null|           null|
|null|        null|120200421085053706|2020-04-18|  ios-phone|          WW| 441004607|   null|             null|               32|       null|           null|
|null|        null|120200421085053706|2020-04-18|  ios-phone|          DE|1053531816|   null|             null|              291|       null|           null|
|null|        null|120200421085053706|2020-04-18|  ios-phone|          WW|1053531816|   null|             null|              293|       null|           null|
|null|        null|120200421085053706|2020-04-18|  ios-phone|          AM|1053531816|   null|             null|                2|       null|           null|
|null|        null|120200421085053706|2020-04-18|    ios-all|          WW|1053531816|   null|             null|              293|       null|           null|
|null|        null|120200421085053706|2020-04-18|    ios-all|          DE|1053531816|   null|             null|              291|       null|           null|
|null|        null|120200421085053706|2020-04-18|    ios-all|          AM|1053531816|   null|             null|                2|       null|           null|
|null|        null|120200421085053706|2020-04-18|    ios-all|          CA| 474591189|   null|             null|                4|       null|           null|
|null|        null|120200421085053706|2020-04-18|    ios-all|          WW| 474591189|   null|             null|                4|       null|           null|
+----+------------+------------------+----------+-----------+------------+----------+-------+-----------------+-----------------+-----------+---------------+

In [0]:
%%sh
PGPASSWORD='TMV!PYT02X*w' psql -h 54.210.244.2 -p 5432 -U app_tomcat -d aa << EOF
\d aa_app;
-- SELECT id, name, company FROM aa_app LIMIT 1;
SELECT id, name, company_id FROM aa_app WHERE id=383004611 LIMIT 1;
EOF

In [0]:

df = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-dna/unified/dna.dna_universal_company_office.v1/dimension/")
df.show(10, False, True)
